## Utils

Conté el codi per descarregar i carregar el model GPT-2 dels repositoris oficials d'OpenAI. A posteriori carregarem els seus pesos, tokenitzador i hiperparàmetre.

In [1]:
import json
import os
import re

import numpy as np
import requests
import tensorflow as tf
from tqdm import tqdm

#Importem el encoder que tenim dins de la carpeta de Modules. (Encoder BPE d'OpenAI GPT-2.)
from modules.encoder import get_encoder

### download_gpt2_files

Funció per descarregar els fitxers de GPT2.

In [ ]:
def download_gpt2_files(model_size, model_dir):
    #Depenent de quina mida de model volguem fer servir tenim per escollir entre ["124M", "355M", "774M", "1558M"]
    assert model_size in ["124M", "355M", "774M", "1558M"]
    url = "https://openaipublic.blob.core.windows.net/gpt-2/models"
    for filename in [
        "checkpoint",
        "encoder.json",
        "hparams.json",
        "model.ckpt.data-00000-of-00001",
        "model.ckpt.index",
        "model.ckpt.meta",
        "vocab.bpe",
    ]:
        r = requests.get(f"{url}/{model_size}/{filename}", stream=True)
        r.raise_for_status()

        with open(os.path.join(model_dir, filename), "wb") as f:
            file_size = int(r.headers["content-length"])
            chunk_size = 1000
            with tqdm(
                ncols=100,
                desc="Fetching " + filename,
                total=file_size,
                unit_scale=True,
                unit="b",
            ) as pbar:
                # 1k for chunk_size, since Ethernet packet size is around 1500 bytes
                for chunk in r.iter_content(chunk_size=chunk_size):
                    f.write(chunk)
                    pbar.update(chunk_size)

In [5]:
#Amb l'assert ens assegurem que la mida que li passem de model està dins de la llista que definim. 
#Aquesta mida de models és la que ens proporciona OpenAI si no controlem això no ens podrem baixar els fitxers ja que la URL serà incorrecte.
#Si posem un model size que no està dins de la llista que definim obtindrem "AssertionError"
model_size="124M"
assert model_size in ["124M", "355M", "774M", "1558M"]


In [6]:
#Tenim els noms de fitxers del Model per baixar-nos emmagatzemats en una variable. Són els que anirem a atacar en la URL.
filenames = ["checkpoint",
        "encoder.json",
        "hparams.json",
        "model.ckpt.data-00000-of-00001",
        "model.ckpt.index",
        "model.ckpt.meta",
        "vocab.bpe"]

In [7]:
#Ens definim la URL del Blob Públic d'Azure d'OpenAI.   
url = "https://openaipublic.blob.core.windows.net/gpt-2/models"

#Per cada fitxer del model a descarregar 
for filename in filenames:
    #Atacarem la URL del Blob Públic amb el recurs en qüestió gràcies a la llibreria requests fent una petició HTTP.
    #Stream=True --> Si no es fa servir aquest paràmetre al fer la petició del request tot el contingut de la resposta es descarregarà inmediatament a la memòria abans de processar-lo. Això pot provocar un s elevat de memòria i causar problemes de rendiment si no es disposa de suficient memòria per manejar la càrrega completa del fitxer.
    #Al fer servir el Stream permet processar el contingut per parts (chunks), reduïnt l'ús de memòria millorant el maneig de grans volums de dades
    r = requests.get(f"{url}/{model_size}/{filename}", stream=True)
    #Comprovem que la petició s'ha fet correctament. Si no fa res la resposta és exitosa (codi 200) de lo contrari llença una excepcióo (per ex 400 o 500)
    r.raise_for_status()


In [8]:
r.raise_for_status

<bound method Response.raise_for_status of <Response [200]>>